In [1]:
import os
import pathlib
import json

In [2]:
import pandas as pd 
import numpy as np

### enter bulletin directory 

In [3]:
bulletin_dir = 'MAM_2019'

In [4]:
dpath = pathlib.Path.cwd().parents[1] / 'bulletin' / bulletin_dir

In [5]:
dpath

PosixPath('/home/nicolasf/operational/ICU/ops/bulletin/MAM_2019')

### reads the country dictionnary for the TRMM rainfall time-series

In [6]:
with open(pathlib.Path.cwd().parents[1] / 'resources' / 'dict_countries.json', 'r') as fj: 
    dict_countries = json.load(fj)

### read the forecast table 

In [7]:
forecast = pd.read_csv(dpath / 'copernicus_forecasts_{}.csv'.format(bulletin_dir), index_col=0)

In [8]:
forecast.head()

,T1,T2,T3,country_fname,std,CONFIDENCE
country_name,,,,,,
New Caledonia,67,18,15,New_Caledonia,13.902801,High
Pitcairn Islands,26,27,47,Pitcairn_Islands,16.699047,Moderate-High
Papua New Guinea,29,30,41,Papua_New_Guinea,13.702688,High
Solomon Islands,25,25,50,Solomon_Islands,17.650556,Moderate-High
Kiribati: Phoenix Islands,14,16,70,Kiribati:_Phoenix_Islands,21.366681,Moderate


In [9]:
def terciles_to_category(df, colnames=['T1','T2','T3'], mindiff=10, climdiff=5): 
    """
    convert a tercile probabilistic forecast into a categorical forecast
    
    if the difference between the maximum probability and the next one is >= mindiff 
    (10 is default) then it's either 'ABOVE', 'NEAR NORMAL' or 'BELOW' depending on the max 
    probability 
    
    if the difference between consecutive probabilities is less than mindiff (10)
    but greater than climdiff (5) then we end up with either 'AVG - BELOW' or 'AVG - ABOVE'
    
    if the difference between consecutive probabilities is less than 5%, it's a CLIMATOLOGY 
    forecasts 
    """
    
    T1, T2, T3 = colnames
    
    cats = []
    for i, row in df.iterrows(): 
        fcst = row.loc[colnames,].astype('int')
        if (fcst.idxmax()  == T3) & ((fcst.loc[T3] - fcst.loc[T2]) >= mindiff): 
            cat = 'ABOVE'
        elif (fcst.idxmax()  == T3) & ((fcst.loc[T3] - fcst.loc[T2]) < mindiff) & ((fcst.loc[T3] - fcst.loc[T2]) > climdiff):
            cat = 'AVG - ABOVE'
        elif (fcst.idxmax()  == T1) & ((fcst.loc[T1] - fcst.loc[T2]) >= mindiff):
            cat = 'BELOW'
        elif (fcst.idxmax()  == T1) & ((fcst.loc[T1] - fcst.loc[T2]) < mindiff) & ((fcst.loc[T1] - fcst.loc[T2]) > climdiff):
            cat = 'AVG - BELOW'
        elif (fcst.idxmax()  == T2) & ((fcst.loc[T2] - fcst.loc[T1]) >= mindiff) & ((fcst.loc[T2] - fcst.loc[T3]) >= mindiff): 
            cat = 'NEAR NORMAL'
        else: 
            cat = 'CLIMATOLOGY'
        cats.append(cat)
    return np.array(cats)

In [10]:
forecast.loc[:,'CATEGORY'] = terciles_to_category(forecast)

In [11]:
forecast

,T1,T2,T3,country_fname,std,CONFIDENCE,CATEGORY
country_name,,,,,,,
New Caledonia,67,18,15,New_Caledonia,13.902801,High,BELOW
Pitcairn Islands,26,27,47,Pitcairn_Islands,16.699047,Moderate-High,ABOVE
Papua New Guinea,29,30,41,Papua_New_Guinea,13.702688,High,ABOVE
Solomon Islands,25,25,50,Solomon_Islands,17.650556,Moderate-High,ABOVE
Kiribati: Phoenix Islands,14,16,70,Kiribati:_Phoenix_Islands,21.366681,Moderate,ABOVE
Fiji,30,31,39,Fiji,17.617841,Moderate-High,AVG - ABOVE
Tuvalu,20,20,60,Tuvalu,15.593552,Moderate-High,ABOVE
Northern Cook Islands,54,24,22,Northern_Cook_Islands,16.716068,Moderate-High,BELOW
Kiribati: Line Islands,3,8,89,Kiribati:_Line_Islands,13.426084,High,ABOVE


### code the categories for the water stress calculations

In [12]:
dict_replace = {}

dict_replace['ABOVE'] = 4

dict_replace['AVG - ABOVE'] = 2
dict_replace['AVG-ABOVE'] = 2
dict_replace['AVG – ABOVE'] = 2
dict_replace['AVG–ABOVE'] = 2

dict_replace['NEAR NORMAL'] = 0
dict_replace['NEAR NORMAL '] = 0
dict_replace['CLIMATOLOGY'] = 0


dict_replace['AVG - BELOW'] = -3
dict_replace['AVG-BELOW'] = -3
dict_replace['AVG – BELOW'] = -3
dict_replace['AVG–BELOW'] = -3

dict_replace['BELOW'] = -5

dict_replace['CLIMATOLOGY'] = -666

In [13]:
forecast = forecast.replace({'CATEGORY':dict_replace})

In [14]:
forecast

,T1,T2,T3,country_fname,std,CONFIDENCE,CATEGORY
country_name,,,,,,,
New Caledonia,67,18,15,New_Caledonia,13.902801,High,-5
Pitcairn Islands,26,27,47,Pitcairn_Islands,16.699047,Moderate-High,4
Papua New Guinea,29,30,41,Papua_New_Guinea,13.702688,High,4
Solomon Islands,25,25,50,Solomon_Islands,17.650556,Moderate-High,4
Kiribati: Phoenix Islands,14,16,70,Kiribati:_Phoenix_Islands,21.366681,Moderate,4
Fiji,30,31,39,Fiji,17.617841,Moderate-High,2
Tuvalu,20,20,60,Tuvalu,15.593552,Moderate-High,4
Northern Cook Islands,54,24,22,Northern_Cook_Islands,16.716068,Moderate-High,-5
Kiribati: Line Islands,3,8,89,Kiribati:_Line_Islands,13.426084,High,4


### changes the names of the columns etc and saves the forecast table for mapping 

In [15]:
table_for_mapping = forecast.copy()

In [16]:
table_for_mapping.loc[:,'COUNTRYNAM'] = table_for_mapping.index

In [17]:
table_for_mapping.rename({'CATEGORY':'Stress'}, axis=1, inplace=True)

In [18]:
table_for_mapping = table_for_mapping.loc[:,['Stress','COUNTRYNAM']]

In [19]:
table_for_mapping.index.name = None

In [20]:
table_for_mapping.to_csv(dpath / 'ICU_forecast_table_for_mapping.csv')

### Now read the last 6 months rainfall categories from TRMM

In [21]:
last_6_months = pd.read_csv(dpath / 'last_6_months_TRMM_categories.csv', index_col=0)

In [22]:
last_6_months.set_index('month', inplace=True)

In [23]:
last_6_months

,Palau,Papua_New_Guinea,Society_Islands,Niue,Kiribati_Phoenix_Islands,Wallis_and_Futuna,Marquesas,Fiji,Kiribati_Line_Islands,Vanuatu_North,...,Tuvalu,Solomon_Islands,Tonga,Pitcairn_Islands,Guam,Northern_Marianas,Federated_States_of_Micronesia,Marshall_Islands,American_Samoa,year
month,,,,,,,,,,,,,,,,,,,,,
9,Below,Well above,Well above,Above,Well below,Normal,Above,Normal,Normal,Well above,...,Above,Well above,Below,Above,Well above,Normal,Well above,Above,Normal,2018
10,Normal,Below,Well below,Below,Above,Well above,Below,Well above,Below,Below,...,Below,Below,Above,Well above,Below,Normal,Well below,Normal,Well above,2018
11,Normal,Normal,Below,Well below,Well above,Below,Normal,Normal,Above,Well below,...,Well above,Well above,Well below,Below,Above,Above,Below,Below,Below,2018
12,Normal,Above,Well below,Well above,Normal,Well above,Above,Well above,Normal,Well above,...,Normal,Well above,Well above,Below,Above,Below,Below,Above,Well above,2018
1,Below,Normal,Above,Above,Normal,Above,Normal,Well above,Above,Below,...,Below,Below,Well above,Well below,Normal,Below,Well above,Normal,Well below,2019
2,Well below,Well below,Normal,Below,Well above,Well above,Well below,Normal,Above,Well above,...,Well above,Above,Well above,Above,Well above,Above,Below,Above,Well above,2019


In [24]:
last_6_months = last_6_months.T

In [25]:
dict_replace = {}
dict_replace['Well below'] = -5 
dict_replace['Below'] = -3 
dict_replace['Normal'] = 0
dict_replace['Above'] = 2
dict_replace['Well above'] = 4

In [26]:
last_6_months.head()

month,9,10,11,12,1,2
Palau,Below,Normal,Normal,Normal,Below,Well below
Papua_New_Guinea,Well above,Below,Normal,Above,Normal,Well below
Society_Islands,Well above,Well below,Below,Well below,Above,Normal
Niue,Above,Below,Well below,Well above,Above,Below
Kiribati_Phoenix_Islands,Well below,Above,Well above,Normal,Normal,Well above


In [27]:
last_6_months  = last_6_months.replace(dict_replace)

In [28]:
last_6_months = last_6_months.sort_index()

In [29]:
last_6_months.columns = range(-6,0)

In [30]:
last_6_months = last_6_months.rename(index=dict_countries)

In [31]:
last_6_months.head()

,-6,-5,-4,-3,-2,-1
American Samoa,0,4,-3,4,-5,4
Austral Islands,2,2,-3,0,4,4
Federated States of Micronesia,4,-5,-3,-3,4,-3
Fiji,0,4,0,4,4,0
Guam,4,-3,2,2,0,4


In [32]:
last_6_months.index.name = 'country_name'

### now calculates the water stress itself 

In [33]:
water_stress = forecast.loc[:,['CATEGORY']]

In [34]:
water_stress = water_stress.merge(last_6_months, left_index=True, right_index=True)

### replace the code for climatology (-666) with 0 (i.e. equivalent to normal) for calculation of the water stress level

In [35]:
water_stress = water_stress.replace({'CATEGORY':{-666:0}})

In [36]:
water_stress.loc[:,'Stress'] = water_stress.loc[:,-1] +  water_stress.loc[:,-2] + \
0.8 * water_stress.loc[:,-3] + \
0.6 * water_stress.loc[:,-4] + \
0.4 * water_stress.loc[:,-5] + \
0.2 * water_stress.loc[:,-6] + \
water_stress.loc[:,'CATEGORY']

In [37]:
water_stress.head()

,CATEGORY,-6,-5,-4,-3,-2,-1,Stress
country_name,,,,,,,,
New Caledonia,-5,0,2,2,4,-5,2,-2.8
Pitcairn Islands,4,2,4,-3,-3,-5,2,-1.2
Papua New Guinea,4,4,-3,0,2,0,-5,0.2
Solomon Islands,4,4,-3,4,4,-3,2,8.2
Kiribati: Phoenix Islands,4,-5,2,4,0,0,4,10.2


### now determine the water stress catergory from the water stress value

In [38]:
bins = [-np.inf, -10, -7, -2, 2, 6, 11, np.inf]

In [39]:
labels = [-3, -2, -1, 0, 1, 2, 3]

In [40]:
water_stress.loc[:,'stress_category'] = pd.cut(water_stress.Stress.values.flatten(), bins=bins, labels=labels)

In [41]:
water_stress

,CATEGORY,-6,-5,-4,-3,-2,-1,Stress,stress_category
country_name,,,,,,,,,
New Caledonia,-5,0,2,2,4,-5,2,-2.8,-1
Pitcairn Islands,4,2,4,-3,-3,-5,2,-1.2,0
Papua New Guinea,4,4,-3,0,2,0,-5,0.2,0
Solomon Islands,4,4,-3,4,4,-3,2,8.2,2
Kiribati: Phoenix Islands,4,-5,2,4,0,0,4,10.2,2
Fiji,2,0,4,0,4,4,0,10.8,2
Tuvalu,4,2,-3,4,0,-3,4,6.6,2
Northern Cook Islands,-5,-3,0,-3,-3,2,2,-5.8,-1
Kiribati: Line Islands,4,0,-3,2,0,2,2,8.0,2


In [42]:
water_stress.index.name = 'locationID'

In [43]:
dpath

PosixPath('/home/nicolasf/operational/ICU/ops/bulletin/MAM_2019')

### now saves for mapping 

In [44]:
water_stress.to_csv(dpath / 'ICU_stress_table_for_mapping.csv')